In [1]:
import spacy
import snscrape.modules.twitter as sntwitter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk

2022-11-07 18:30:26.867906: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-07 18:30:26.867984: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def extractor(query: str, csvname: str):
    tweets = []

    for tweet in sntwitter.TwitterSearchScraper(query=query).get_items():
        tweets.append(
            [
                tweet.date,
                tweet.id,
                tweet.content,
                tweet.user.username,
            ]
        )
    df = pd.DataFrame(tweets, columns=["Datetime", "Tweet Id", "Text", "Username"])
    df["Processed Text"] = [re.sub(r"http\S+", "", tweet) for tweet in df["Text"]]
    df["Processed Text"] = [re.sub(r"[@]", "", x) for x in df["Processed Text"]]
    df["Processed Text"] = [re.sub(r"#", "", x) for x in df["Processed Text"]]
    df["Processed Text"] = [re.sub(r"\n|\t|\r", " ", x) for x in df["Processed Text"]]

    df.to_parquet(csvname,engine='fastparquet',compression='gzip')


In [3]:
extractor(
    "#Vasco lang:pt until:2022-11-06 since:2022-11-05",
    "data_vasco.parquet",
)

In [4]:
df = pd.read_parquet("data_vasco.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   Datetime        80 non-null     datetime64[ns, UTC]
 1   Tweet Id        80 non-null     int64              
 2   Text            80 non-null     object             
 3   Username        80 non-null     object             
 4   Processed Text  80 non-null     object             
dtypes: datetime64[ns, UTC](1), int64(1), object(3)
memory usage: 3.2+ KB


In [18]:
def strip_emoji(text):
        emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
        return emoji_pattern.sub('EMOJI', text)

In [19]:
df['Emoji Tag'] = df['Processed Text'].apply(strip_emoji)

In [20]:
df.head(30)

,Datetime,Tweet Id,Text,Username,Processed Text,Emoji Tag
0,2022-11-05 23:51:19+00:00,1589042707963334656,Amanhã está hora.......\n#VASCO,Felipevascaino7,Amanhã está hora....... VASCO,Amanhã está hora....... VASCO
1,2022-11-05 23:39:48+00:00,1589039808877592576,Sempre esteja conosco.\nPrecisamos do senhor a...,VSF69666,Sempre esteja conosco. Precisamos do senhor am...,Sempre esteja conosco. Precisamos do senhor am...
2,2022-11-05 23:37:43+00:00,1589039287542067200,Disse o presidente do DIM sobre possível inter...,Ruanndelima,Disse o presidente do DIM sobre possível inter...,Disse o presidente do DIM sobre possível inter...
3,2022-11-05 23:35:27+00:00,1589038717226086401,"Segundo o companheiro @JuanPabloRuaJim, da @Ra...",Felipe_Bocha,"Segundo o companheiro JuanPabloRuaJim, da Radi...","Segundo o companheiro JuanPabloRuaJim, da Radi..."
4,2022-11-05 22:49:55+00:00,1589027258068766720,Tentando esquecer que amanhã tem #VASCO https:...,chuckyfw7,Tentando esquecer que amanhã tem VASCO,Tentando esquecer que amanhã tem VASCO
5,2022-11-05 22:38:44+00:00,1589024444705804288,Queria deixar um recado do fundo do meu coraçã...,Biamoraiis03,Queria deixar um recado do fundo do meu coraçã...,Queria deixar um recado do fundo do meu coraçã...
6,2022-11-05 22:28:33+00:00,1589021880052494336,Bora dormir agora e acordar só segunda-feira!!...,prchome99,Bora dormir agora e acordar só segunda-feira!!...,Bora dormir agora e acordar só segunda-feira!!...
7,2022-11-05 22:22:51+00:00,1589020447748001793,@lelebmachado O VASCO SOBE! \n#vasco,daniirichter10,lelebmachado O VASCO SOBE! vasco,lelebmachado O VASCO SOBE! vasco
8,2022-11-05 22:04:25+00:00,1589015805660139520,Vendo dois ingressos pro jogo #ItuanoXVasco \n...,BessaRodri,Vendo dois ingressos pro jogo ItuanoXVasco V...,Vendo dois ingressos pro jogo ItuanoXVasco V...
9,2022-11-05 21:42:07+00:00,1589010193203232768,"🟢⚪️Lembra dele? Ex-Palmeiras e Vasco, 'Rei da ...",pimentelmidia,"🟢⚪️Lembra dele? Ex-Palmeiras e Vasco, 'Rei da ...","🟢⚪️Lembra dele? Ex-Palmeiras e Vasco, 'Rei da ..."
